In [1]:
import os
import time
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import cv2
from tqdm import tqdm

import multiprocessing as mp
from joblib import Parallel, delayed

from utils import vrrotvec2mat
from distance_to_coordinates_triangle_IJ import distance_to_coordinates_triangle_IJ_parallel

from utils import save_intermediate_results, load_intermediate_results, generate_savename, plot_triangle

# ========== Parameter Initialization ==========

In [2]:
parameter_case_id = "cos_Run1"

In [3]:
# Run Parameters
redo = False
n_jobs = 16 #12
if n_jobs == -1:
    n_jobs = mp.cpu_count()
verbose = 0 #1 #0
long_trajectory = True

# Simulation Global Parameters
runLabel = '3VarMassUUD'
funcType = 'cos' #'sin'
varType = '3Var-all-Pub-Painleve-test'
transform = 'XYMC'

# Trajectory Parameters
# ----------------------------------------------------------------------------------
if parameter_case_id == "cos_Run1":
    # Run 202504-Production Run1
    mass = np.array([2.1, 2.1, 4.7])
    d0 = np.array([1.1, 1.0, 1.0])
    aij = np.array([0.20, 0.15, 0.15])
    ratio_Tdelta = 100 if long_trajectory else 1000 #100 #1000
    Tij = np.array([700*ratio_Tdelta, 2100*ratio_Tdelta, 2100*ratio_Tdelta])
    deltaT = (1/ratio_Tdelta)*1e-3
    Phiij = np.array([0, np.pi/4, -np.pi/4])
# ----------------------------------------------------------------------------------

# Time Parameters
total_periods = 200 if long_trajectory else 20 #200 #20
stepsLeastCommonPeriod = np.lcm.reduce(Tij.astype(int))  # steps of least common period
totalT = total_periods * stepsLeastCommonPeriod * deltaT  # total time of trajectory
totalStep = int(np.ceil(totalT / deltaT)) # number of steps
steps = np.arange(0, totalStep + 1)

# --- Print Information ---
print("--- Run info ---")
print("runLabel : {:}".format(runLabel))
print("funcType : {:}".format(funcType))
print("varType  : {:}".format(varType))
print("totalT   : {:7.4e}".format(totalT))
print("totalStep: {:d}".format(totalStep))
print("deltaT   : {:g}".format(deltaT))

print("mass: {}".format(mass))
print("d0  : {}".format(d0))
print("aij : {}".format(aij))
print("Tij : {}".format(Tij * deltaT))
print("Phij: {}".format(Phiij))
print("Least common period: {:.4g} [{:d}]".format(deltaT * stepsLeastCommonPeriod, stepsLeastCommonPeriod))
print("----------------")

--- Run info ---
runLabel : 3VarMassUUD
funcType : cos
varType  : 3Var-all-Pub-Painleve-test
totalT   : 4.2000e+02
totalStep: 42000000
deltaT   : 1e-05
mass: [2.1 2.1 4.7]
d0  : [1.1 1.  1. ]
aij : [0.2  0.15 0.15]
Tij : [0.7 2.1 2.1]
Phij: [ 0.          0.78539816 -0.78539816]
Least common period: 2.1 [210000]
----------------


In [4]:
scale_factor = np.mean((Tij*deltaT)/(2*np.pi))/(1e-23)

scale_factor, np.mean((Tij*deltaT)/(2*np.pi))

(np.float64(2.5995307371676243e+22), np.float64(0.2599530737167624))

In [5]:
save_data = {
    'runLabel': runLabel,
    'funcType': funcType,
    'varType': varType,
    'transform': transform,
    'mass': mass,
    'd0': d0,
    'aij': aij,
    'ratio_Tdelta': ratio_Tdelta,
    'Tij': Tij,
    'deltaT': deltaT,
    'Phiij': Phiij,
    'total_periods': total_periods
}

In [6]:
save_dir = "./data"
save_name = generate_savename(save_data)
save_data_exists = os.path.exists(os.path.join(save_dir, save_name))

save_dir, save_name, save_data_exists

('./data',
 '3VarMassUUD_cos_e8ed382a034d5eb0f2787073dba3803d0387b66f4e1f85c2feb492b768fb4711',
 True)

In [7]:
redo = redo or (not save_data_exists)
redo

False

In [8]:
if not redo:
    print("[Load Data from {:}]".format(os.path.join(save_dir, save_name)))
    start_time = time.time()
    
    save_data = load_intermediate_results(os.path.join(save_dir, save_name))
    print("Time Cost: {:.2f}s".format(time.time() - start_time))

[Load Data from ./data/3VarMassUUD_cos_e8ed382a034d5eb0f2787073dba3803d0387b66f4e1f85c2feb492b768fb4711]
Time Cost: 18.81s


In [9]:
print(save_data.keys())

dict_keys(['runLabel', 'funcType', 'varType', 'transform', 'mass', 'd0', 'aij', 'ratio_Tdelta', 'Tij', 'deltaT', 'Phiij', 'total_periods', 'dij', 'Theta', 'S', 'R'])


# ========== Generate Distance Matrix `dij` ==========

In [10]:
if redo or ('dij' not in save_data.keys()):
    # dinstance matrix: `dij`
    edgeN = 3
    dij = np.zeros((edgeN, len(steps)))
    for i in tqdm(range(edgeN), desc="Generating dij", unit="step"):
        if funcType == "sin":
            dij[i, :] = d0[i] + aij[i] * np.sin(2 * np.pi * steps / Tij[i] + Phiij[i])
        if funcType == "cos":
            dij[i, :] = d0[i] + aij[i] * np.cos(2 * np.pi * steps / Tij[i] + Phiij[i])
else:
    print('dij is already in save_data!')
    dij = save_data['dij']
    print(dij.shape)

print("Memory of dij     : {:9.3f} MB".format(dij.nbytes/(1024**2)))

dij is already in save_data!
(3, 42000001)
Memory of dij     :   961.304 MB


# ========== Main ==========

## --- Convert `dij` to `S` ---

In [11]:
if redo or ('S' not in save_data.keys()):
    print("[Step: dij -> S]")
    start_time = time.time()
    S, ResDiff, _, _ = distance_to_coordinates_triangle_IJ_parallel(dij, transform, mass, n_jobs=n_jobs, verbose=verbose)
    print("Time Cost: {:.2f}s".format(time.time() - start_time))
    print("ResDiff: {:.2e} [{:.3e}]".format(np.mean(ResDiff), np.std(ResDiff)))
else:
    print('S is already in save_data!')
    S = save_data['S']
    print(S.shape)

print("Memory of S       : {:9.3f} MB".format(S.nbytes/(1024**2)))

S is already in save_data!
(3, 3, 42000001)
Memory of S       :  2883.911 MB


## --- Calculate `Sdot` ---

In [12]:
if redo or ('Sdot' not in save_data.keys()):
    print("[Step: S -> Sdot]")
    start_time = time.time()
    Sdot = np.full_like(S, np.nan)
    Sdot[:,:,:-1] = (S[:,:,1:] - S[:,:,:-1])/deltaT
    
    print("Time Cost: {:.2f}s".format(time.time() - start_time))
else:
    print('Sdot is already in save_data!')
    Sdot = save_data['Theta']
    print(Sdot.shape)

print("Memory of Sdot    : {:9.3f} MB".format(Sdot.nbytes/(1024**2)))

[Step: S -> Sdot]
Time Cost: 1.33s
Memory of Sdot    :  2883.911 MB


## --- Calculate `Thetadot` ---

In [13]:
if redo or ('Thetadot' not in save_data.keys()):
    print("[Step: (S,Sdot) -> Thetadot]")
    start_time = time.time()
    Thetadot = np.zeros(S.shape[2])

    def compute_Thetadot(t):
        numerator = np.sum(mass * (S[:, 1, t] * Sdot[:, 0, t] - S[:, 0, t] * Sdot[:, 1, t]))
        denominator = np.sum(mass * (S[:, 0, t]**2 + S[:, 1, t]**2))
        return numerator / denominator if denominator != 0 else 0
    
    with Parallel(n_jobs=n_jobs,verbose=verbose) as parallel:
        Thetadot = parallel(delayed(compute_Thetadot)(t) for t in range(S.shape[2]))
        Thetadot = np.array(Thetadot)
    
    print("Time Cost: {:.2f}s".format(time.time() - start_time))
else:
    print('Thetadot is already in save_data!')
    Thetadot = save_data['Thetadot']
    print(Thetadot.shape)

print("Memory of Thetadot: {:9.3f} MB".format(Thetadot.nbytes/(1024**2)))

[Step: (S,Sdot) -> Thetadot]
Time Cost: 189.01s
Memory of Thetadot:   320.435 MB


## --- Integrate `Theta` ---

In [14]:
if redo or ('Theta' not in save_data.keys()):
    print("[Step: Thetadot -> Theta]")
    start_time = time.time()
    Theta = np.zeros_like(Thetadot)

    for t in tqdm(range(1, len(Theta)), desc="Calculating Theta", unit="step"):
        Theta[t] = Theta[t-1] + Thetadot[t-1] * deltaT

    print("Time Cost: {:.2f}s".format(time.time() - start_time))
else:
    print('Theta is already in save_data!')
    Theta = save_data['Theta']
    print(Theta.shape)

print("Memory of Theta   : {:9.3f} MB".format(Theta.nbytes/(1024**2)))

Theta is already in save_data!
(42000001,)
Memory of Theta   :   320.435 MB


## --- Rotate to `R` ---

In [15]:
if redo or ('R' not in save_data.keys()):
    print("[Step: (S,Theta) -> R]")
    start_time = time.time()
    R = np.full_like(S, np.nan)

    def compute_rotation(t):
        rotvec = np.array([0, 0, 1, Theta[t]])
        rotmat = vrrotvec2mat(rotvec)
        return S[:, :, t] @ rotmat.T

    with Parallel(n_jobs=n_jobs,verbose=verbose,return_as="generator") as parallel:
        results = parallel(delayed(compute_rotation)(t) for t in range(S.shape[2]))
        with tqdm(total=S.shape[2], desc="Calculating R", unit="step") as pbar:
            for t, result in enumerate(results):
                R[:, :, t] = result
                pbar.update(1)

    print("Time Cost: {:.2f}s".format(time.time() - start_time))
else:
    print('R is already in save_data!')
    R = save_data['R']
    print(R.shape)

print("Memory of R       : {:9.3f} MB".format(R.nbytes/(1024**2)))

R is already in save_data!
(3, 3, 42000001)
Memory of R       :  2883.911 MB


# ========== Save Results ==========

In [16]:
if redo or any([v not in save_data for v in ['dij', 'S', 'R', 'Theta']]):
    save_data.update({
        'dij': dij, # distance matrix
        'Theta': Theta, # angle
        'S': S, # coordinates in shape space
        'R': R, # coordinates in real space
    })
    
    save_intermediate_results(save_data, save_name=save_name, save_dir=save_dir)
else:
    print("All variables exist in save_data!")

All variables exist in save_data!


In [17]:
print("Memory of dij     : {:9.3f} MB {:}".format(dij.nbytes/(1024**2), dij.shape))
print("Memory of Theta   : {:9.3f} MB {:}".format(Theta.nbytes/(1024**2), Theta.shape))
print("Memory of Thetadot: {:9.3f} MB {:}".format(Thetadot.nbytes/(1024**2), Thetadot.shape))
print("Memory of S       : {:9.3f} MB {:}".format(S.nbytes/(1024**2), S.shape))
print("Memory of Sdot    : {:9.3f} MB {:}".format(Sdot.nbytes/(1024**2), Sdot.shape))
print("Memory of R       : {:9.3f} MB {:}".format(R.nbytes/(1024**2), R.shape))

Memory of dij     :   961.304 MB (3, 42000001)
Memory of Theta   :   320.435 MB (42000001,)
Memory of Thetadot:   320.435 MB (42000001,)
Memory of S       :  2883.911 MB (3, 3, 42000001)
Memory of Sdot    :  2883.911 MB (3, 3, 42000001)
Memory of R       :  2883.911 MB (3, 3, 42000001)


# ========== Print Results ==========

In [18]:
print("----------------")
print("DeltaTheta = {:<.8g} rad".format(Theta[-1]-Theta[0]))

nPeriod = totalT/(stepsLeastCommonPeriod*deltaT)
print("nPeriod = {:<.8g}".format(nPeriod))
if np.abs(nPeriod - np.round(nPeriod)) < 1e-12:
    print("DeltaTheta/nPeriod = {:<.12g} rad/period".format((Theta[-1]-Theta[0])/nPeriod))
print("----------------")

----------------
DeltaTheta = -10.72389 rad
nPeriod = 200
DeltaTheta/nPeriod = -0.0536194517426 rad/period
----------------


# ========== Visualize Results ==========

In [19]:
figure_dir = os.path.join(".", "figures", save_name)
os.makedirs(figure_dir, exist_ok=True)

In [20]:
plt.rc('text', usetex=False)
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'axes.linewidth': 1.5,
    'lines.linewidth': 2,
    'legend.fontsize': 12
})

In [21]:
def plot_period_vlines(ax, 
                       x_values,
                       y_values=None,
                       color='k', alpha=0.5,
                       linestyle='--', linewidth=1.5,
                       marker='o', markersize=8, markerfacecolor='k', 
                       markerlinestyle='--', markerlinewidth=2, markerlinecolor='k',
                       show_vline=True, show_markline=True):
    if y_values is not None:
        x_period = []
        y_period = []
    for i_period in range(total_periods+1):
        x_vline = x_values[0] + i_period*stepsLeastCommonPeriod*deltaT
        if x_vline <= x_values[-1]:
            if show_vline:
                ax.axvline(x_vline, color=color, linestyle=linestyle, linewidth=linewidth, alpha=alpha)
            #
            if y_values is not None:
                x_period.append((steps[0]+i_period*stepsLeastCommonPeriod)*deltaT)
                y_period.append(y_values[steps[0]+i_period*stepsLeastCommonPeriod])
        else:
            break
    if y_values is not None and show_markline:
        ax.plot(x_period, y_period, markerlinestyle, 
                linewidth=markerlinewidth, color=markerlinecolor,
                marker=marker, markersize=markersize, markerfacecolor=markerfacecolor)

## Visualizations-1

In [22]:
index_plot = range(0,steps.shape[-1],100)

index_plot, steps[index_plot].shape, steps[index_plot]

(range(0, 42000001, 100),
 (420001,),
 array([       0,      100,      200, ..., 41999800, 41999900, 42000000],
       shape=(420001,)))

### **Figures 1 a)**: Trajectories

In [23]:
def plot_trajectories(figure_dir, index_plot=None,
                      figure_name='trajectories.png', figsize=(20,10), dpi=150, 
                      show_ref_line=True, axis_lim=[0.9,0.9], axis_tick_interval=[0.3,0.3],
                      linestyle='--', linewidth=1.5, 
                      show_marker=False, markersize=8, markerfacecolor=None, markeredgecolor='k',
                      show_text=True, text_prefix="P", text_fontsize=20, text_color='', text_offset=[[0, 0],[0,0]],
                      show_edge=True, edge_offset=0.01):
    fig = plt.figure(figsize=figsize)
    
    colors = [[0, 100/255, 0], [1, 0, 0], [0, 0, 1]]  # RGB颜色
    
    x_lim = np.array([-1,1])*axis_lim[0]
    x_ticks_pos = np.arange(int(np.ceil(x_lim[0]/axis_tick_interval[0])), int(np.floor(x_lim[1]/axis_tick_interval[0]))+1)*axis_tick_interval[0]
    y_lim = np.array([-1,1])*axis_lim[1]
    y_ticks_pos = np.arange(int(np.ceil(y_lim[0]/axis_tick_interval[1])), int(np.floor(y_lim[1]/axis_tick_interval[1]))+1)*axis_tick_interval[1]
    
    if index_plot is None:
        index_plot = range(0, steps.shape[-1])
    
    # Shape space
    ax1 = fig.add_subplot(121)
    # 
    if show_ref_line:
        ax1.axhline(0, color='k', linestyle='--', linewidth=1.5)
        ax1.axvline(0, color='k', linestyle='--', linewidth=1.5)
    for i in range(3):
        ax1.plot(S[i,0,index_plot], S[i,1,index_plot], color=colors[i], linewidth=2)
    plot_triangle(S[:,:,index_plot[-1]], ax1, 'k', 'Shape',
                  linestyle=linestyle, linewidth=linewidth,
                  show_marker=show_marker, markersize=markersize, markerfacecolor=markerfacecolor, markeredgecolor=markeredgecolor,
                  show_text=show_text, text_prefix=text_prefix, text_fontsize=text_fontsize, text_color=text_color, text_offset=text_offset[0],
                  show_edge=show_edge, edge_offset=edge_offset)
    ax1.set_title('Reference Triangle')
    ax1.grid(False)
    ax1.set_xlim(x_lim)
    ax1.set_ylim(y_lim)
    ax1.set_xticks(x_ticks_pos)
    ax1.set_yticks(y_ticks_pos)
    ax1.set_aspect('equal', 'box')
    
    # Real Space
    ax2 = fig.add_subplot(122)
    #
    if show_ref_line:
        ax2.axhline(0, color='k', linestyle='--', linewidth=1.5)
        ax2.axvline(0, color='k', linestyle='--', linewidth=1.5)
    for i in range(3):
        ax2.plot(R[i,0,index_plot], R[i,1,index_plot], color=colors[i], linewidth=2)
    plot_triangle(R[:,:,index_plot[-1]], ax2, 'k', 'Real',
                  linestyle=linestyle, linewidth=linewidth,
                  show_marker=show_marker, markersize=markersize, markerfacecolor=markerfacecolor, markeredgecolor=markeredgecolor,
                  show_text=show_text, text_prefix=text_prefix, text_fontsize=text_fontsize, text_color=text_color, text_offset=text_offset[1],
                  show_edge=show_edge, edge_offset=edge_offset)
    ax2.set_title('Physical Triangle')
    ax2.grid(False)
    ax2.set_xlim(x_lim)
    ax2.set_ylim(y_lim)
    ax2.set_xticks(x_ticks_pos)
    ax2.set_yticks(y_ticks_pos)
    ax2.set_aspect('equal', 'box')

    plt.tight_layout()
    plt.savefig(os.path.join(figure_dir,figure_name), dpi=dpi)
    plt.close()

In [24]:
plot_trajectories(figure_dir, index_plot=range(0,70*stepsLeastCommonPeriod+1,100), 
                  figsize=(10,5), figure_name='trajectories.png', show_ref_line=True,
                  linestyle='-', linewidth=3,
                  show_marker=True, markersize=15, markerfacecolor=[[0, 100/255, 0], [1, 0, 0], [0, 0, 1]], markeredgecolor='k',
                  show_edge=False, text_prefix="V", text_color='k', text_offset=[[[-0.08,0.04],[-0.20,-0.01],[-0.05,-0.2]],[[-0.06,0.04],[-0.24,-0.1],[0.017,-0.15]]])

## Visualization-2

### Preparation

In [25]:
index_plot = range(0,5*stepsLeastCommonPeriod+1,1) if long_trajectory else index_plot

index_plot, steps[index_plot].shape, steps[index_plot]

(range(0, 1050001),
 (1050001,),
 array([      0,       1,       2, ..., 1049998, 1049999, 1050000],
       shape=(1050001,)))

### Velocities

#### Time Derivatives of S

In [26]:
# Time Derivatives of S
print("Time Derivatives of S")
start_time = time.time()

# Sdot
SdotNorm = np.sqrt(np.einsum('ijk,ijk->ik', Sdot, Sdot))

print("Time Cost: {:.2f}s".format(time.time() - start_time))
print("Memory of S       : {:9.3f} MB {:}".format(S.nbytes/(1024**2), S.shape))
print("Memory of Sdot    : {:9.3f} MB {:}".format(Sdot.nbytes/(1024**2), Sdot.shape))
print("Memory of SdotNorm: {:9.3f} MB {:}".format(SdotNorm.nbytes/(1024**2), SdotNorm.shape))

Time Derivatives of S
Time Cost: 0.55s
Memory of S       :  2883.911 MB (3, 3, 42000001)
Memory of Sdot    :  2883.911 MB (3, 3, 42000001)
Memory of SdotNorm:   961.304 MB (3, 42000001)


#### Time Derivatives of R

In [27]:
# Time Derivatives of R
print("Time Derivatives of R")
start_time = time.time()

# Rdot
Rdot = np.full_like(R, np.nan)
Rdot[:,:,:-1] = (R[:,:,1:] - R[:,:,:-1])/deltaT
# RdotNorm
RdotNorm = np.sqrt(np.einsum('ijk,ijk->ik', Rdot, Rdot))

print("Time Cost: {:.2f}s".format(time.time() - start_time))
print("Memory of R       : {:9.3f} MB {:}".format(R.nbytes/(1024**2), R.shape))
print("Memory of Rdot    : {:9.3f} MB {:}".format(Rdot.nbytes/(1024**2), Rdot.shape))
print("Memory of RdotNorm: {:9.3f} MB {:}".format(RdotNorm.nbytes/(1024**2), RdotNorm.shape))

Time Derivatives of R
Time Cost: 1.89s
Memory of R       :  2883.911 MB (3, 3, 42000001)
Memory of Rdot    :  2883.911 MB (3, 3, 42000001)
Memory of RdotNorm:   961.304 MB (3, 42000001)


#### Time Reversal Point

In [28]:
from utils import get_extremes

In [29]:
print("== Shape space ==")
idx_min_SumSdotNorm, value_min_SumSdotNorm = get_extremes(np.sum(SdotNorm,axis=0), "min", 1e-4)
idx_max_SumSdotNorm, value_max_SumSdotNorm = get_extremes(np.sum(SdotNorm,axis=0), "max", 1e-4)

print("== Real space ==")
idx_min_SumRdotNorm, value_min_SumRdotNorm = get_extremes(np.sum(RdotNorm,axis=0), "min", 1e-4)
idx_max_SumRdotNorm, value_max_SumRdotNorm = get_extremes(np.sum(RdotNorm,axis=0), "max", 1e-4)

== Shape space ==
Minimal value = 0.44774724
Number of minimal values: [18800]
Maximal value = 3.42866006
Number of maximal values: [36400]
== Real space ==
Minimal value = 0.45132822
Number of minimal values: [18000]
Maximal value = 2.52544060
Number of maximal values: [86400]


### Momentum and Angular Momentum

#### Momentum in Shape Space and Real Space

In [30]:
print("Momentum in Shape Space and Real Space")
start_time = time.time()

momentum_shape = np.sum(mass[:,np.newaxis,np.newaxis] * Sdot, axis=0)
momentum_real = np.sum(mass[:,np.newaxis,np.newaxis] * Rdot, axis=0)

print("Time Cost: {:.2f}s".format(time.time() - start_time))
print("Memory of momentum_shape: {:9.3f} MB {:}".format(momentum_shape.nbytes/(1024**2), momentum_shape.shape))
print("Memory of momentum_real : {:9.3f} MB {:}".format(momentum_real.nbytes/(1024**2), momentum_real.shape))

Momentum in Shape Space and Real Space
Time Cost: 1.39s
Memory of momentum_shape:   961.304 MB (3, 42000001)
Memory of momentum_real :   961.304 MB (3, 42000001)


In [31]:
print("Momentum in shape space [avg <std>]: {:} <{:}>".format(np.nanmean(momentum_shape, axis=1), np.nanstd(momentum_shape, axis=1)))
print("Momentum in real  space [avg <std>]: {:} <{:}>".format(np.nanmean(momentum_real, axis=1), np.nanstd(momentum_real, axis=1)))

Momentum in shape space [avg <std>]: [-5.89944528e-17 -4.38148970e-18  0.00000000e+00] <[6.73325647e-11 5.92087666e-11 0.00000000e+00]>
Momentum in real  space [avg <std>]: [-3.69347410e-18 -1.46879153e-18  0.00000000e+00] <[6.55080541e-11 6.62943294e-11 0.00000000e+00]>


#### Angular Momentum in Shape Space and Real Space

In [32]:
from utils import compute_angular_momentum

In [33]:
print("Angular Momentum in Shape Space and Real Space")
start_time = time.time()

angular_momentum_shape = compute_angular_momentum(S, Sdot, mass)
angular_momentum_real = compute_angular_momentum(R, Rdot, mass)

print("Time Cost: {:.2f}s".format(time.time() - start_time))
print("Memory of angular_momentum_shape: {:9.3f} MB {:}".format(angular_momentum_shape.nbytes/(1024**2), angular_momentum_shape.shape))
print("Memory of angular_momentum_real : {:9.3f} MB {:}".format(angular_momentum_real.nbytes/(1024**2), angular_momentum_real.shape))

Angular Momentum in Shape Space and Real Space
Time Cost: 4.18s
Memory of angular_momentum_shape:   961.304 MB (3, 42000001)
Memory of angular_momentum_real :   961.304 MB (3, 42000001)


In [34]:
print("Angular momentum in shape space [avg <std>]: {:} <{:}>".format(np.nanmean(angular_momentum_shape, axis=1), np.nanstd(angular_momentum_shape, axis=1)))
print("Angular momentum in real  space [avg <std>]: {:} <{:}>".format(np.nanmean(angular_momentum_real, axis=1), np.nanstd(angular_momentum_real, axis=1)))

Angular momentum in shape space [avg <std>]: [0.         0.         0.09964652] <[0.         0.         2.41459523]>
Angular momentum in real  space [avg <std>]: [ 0.00000000e+00  0.00000000e+00 -4.86627356e-06] <[0.00000000e+00 0.00000000e+00 5.36091072e-06]>


#### Inertia Tensoer and Instantaneous Angular Velocity

In [35]:
from utils import compute_inertia_tensor, compute_angular_velocity

In [36]:
print("Inertia Tensor in Shape Space and Real Space")
start_time = time.time()

inertia_tensor_shape = compute_inertia_tensor(S, mass)
inertia_tensor_real = compute_inertia_tensor(R, mass)

print("Time Cost: {:.2f}s".format(time.time() - start_time))
print("Memory of inertia_tensor_shape: {:9.3f} MB {:}".format(inertia_tensor_shape.nbytes/(1024**2), inertia_tensor_shape.shape))
print("Memory of inertia_tensor_real : {:9.3f} MB {:}".format(inertia_tensor_real.nbytes/(1024**2), inertia_tensor_real.shape))

Inertia Tensor in Shape Space and Real Space
Time Cost: 24.86s
Memory of inertia_tensor_shape:  2883.911 MB (3, 3, 42000001)
Memory of inertia_tensor_real :  2883.911 MB (3, 3, 42000001)


In [37]:
print("Average of inertia tensor in shape space:")
print(np.nanmean(inertia_tensor_shape, axis=2))
print(np.nanstd(inertia_tensor_shape, axis=2))
print("Average of inertia tensor in real  space:")
print(np.nanmean(inertia_tensor_real, axis=2))
print(np.nanstd(inertia_tensor_real, axis=2))

Average of inertia tensor in shape space:
[[ 1.39113931 -0.13359315  0.        ]
 [-0.13359315  1.4612624   0.        ]
 [ 0.          0.          2.8524017 ]]
[[0.29999813 0.29651736 0.        ]
 [0.29651736 0.29127225 0.        ]
 [0.         0.         0.36674659]]
Average of inertia tensor in real  space:
[[ 1.41638826 -0.00650134  0.        ]
 [-0.00650134  1.43601345  0.        ]
 [ 0.          0.          2.8524017 ]]
[[0.33609308 0.28309761 0.        ]
 [0.28309761 0.33957649 0.        ]
 [0.         0.         0.36674659]]


In [38]:
print("Instantaneous Angular Velocity in Shape Space and Real Space")
start_time = time.time()

omega_shape = compute_angular_velocity(inertia_tensor_shape, angular_momentum_shape)
omega_real = compute_angular_velocity(inertia_tensor_real, angular_momentum_real)

print("Time Cost: {:.2f}s".format(time.time() - start_time))
print("Memory of omega_shape: {:9.3f} MB {:}".format(omega_shape.nbytes/(1024**2), omega_shape.shape))
print("Memory of omega_real : {:9.3f} MB {:}".format(omega_real.nbytes/(1024**2), omega_real.shape))

Instantaneous Angular Velocity in Shape Space and Real Space
Time Cost: 7.47s
Memory of omega_shape:   961.304 MB (3, 42000001)
Memory of omega_real :   961.304 MB (3, 42000001)


In [39]:
print("Rotation Angle via Integrating omega over All      in Shape Space: {:}".format(np.nansum(omega_shape, axis=-1)*deltaT))
print("Rotation Angle via Integrating omega over 1-Period in Shape Space: {:}".format(np.nansum(omega_shape, axis=-1)*deltaT/total_periods))

print("Rotation Angle via Integrating omega over All      in Real  Space: {:}".format(np.nansum(omega_real, axis=-1)*deltaT))
print("Rotation Angle via Integrating omega over 1-Period in Real  Space: {:}".format(np.nansum(omega_real, axis=-1)*deltaT/total_periods))

Rotation Angle via Integrating omega over All      in Shape Space: [ 0.          0.         10.72389035]
Rotation Angle via Integrating omega over 1-Period in Shape Space: [0.         0.         0.05361945]
Rotation Angle via Integrating omega over All      in Real  Space: [ 0.          0.         -0.00073567]
Rotation Angle via Integrating omega over 1-Period in Real  Space: [ 0.00000000e+00  0.00000000e+00 -3.67835721e-06]


In [40]:
print("Theta(T)_connection: {:>18.12f}".format((Theta[-1]-Theta[0])/total_periods))
print("Theta(T)_omegashape: {:>18.12f}".format((np.nansum(omega_shape, axis=-1)*deltaT/total_periods)[-1]))
print("Theta(T)_omegareal : {:>18.12f}".format((np.nansum(omega_real, axis=-1)*deltaT/total_periods)[-1]))

Theta(T)_connection:    -0.053619451743
Theta(T)_omegashape:     0.053619451743
Theta(T)_omegareal :    -0.000003678357


In [41]:
print("[omega -> Theta_omega]")
start_time = time.time()
Theta_omega_shape = np.zeros_like(omega_shape)
Theta_omega_real = np.zeros_like(omega_real)

for t in tqdm(range(1, len(steps)), desc="Calculating Theta_omega", unit="step"):
    Theta_omega_shape[:,t] = Theta_omega_shape[:,t-1] + omega_shape[:,t-1]*deltaT
    Theta_omega_real[:,t] = Theta_omega_real[:,t-1] + omega_real[:,t-1]*deltaT

print("Time Cost: {:.2f}s".format(time.time() - start_time))

[omega -> Theta_omega]


Calculating Theta_omega: 100%|██████████| 42000000/42000000 [02:01<00:00, 346748.51step/s]

Time Cost: 121.31s


#### **Figures 1 b)**: Angule and Effective angular momentum

In [42]:
inertia_tensor_real_average = np.nanmean(inertia_tensor_real, axis=2)
inertia_tensor_real_average

array([[ 1.41638826, -0.00650134,  0.        ],
       [-0.00650134,  1.43601345,  0.        ],
       [ 0.        ,  0.        ,  2.8524017 ]])

In [43]:
angular_momentum_eff = inertia_tensor_real_average[2,2]*Theta/(steps*deltaT)
angular_momontum_eff_avg = np.nanmean(angular_momentum_eff[100*stepsLeastCommonPeriod:])

/tmp/ipykernel_2850168/4268061049.py:1: RuntimeWarning: invalid value encountered in divide
  angular_momentum_eff = inertia_tensor_real_average[2,2]*Theta/(steps*deltaT)


In [44]:
print("Averaged moment of inertia: {:.8f}".format(inertia_tensor_real_average[2,2]))
print("Averaged effective angular momentum: {:.8e}".format(angular_momontum_eff_avg))

Averaged moment of inertia: 2.85240170
Averaged effective angular momentum: -7.23839659e-02


In [45]:
def plot_combined_theta_angular(figure_dir, index_plot=None,
                                figure_name="combined_theta_angular.png", figsize=(12,10), dpi=150,
                                theta_linewidth=3, angular_linewidth=3,
                                theta_color='b', angular_color='r',
                                xlim=None, xtick_spacing=None, theta_ylim=None, angular_ylim=None, 
                                show_zeroline=True, show_vlines=True, show_marklines=True,
                                label_theta='$\\theta$', label_angular='$L_{eff}$', title_string=None):
    #
    fig, ax1 = plt.subplots(figsize=figsize)
    ax2 = ax1.twinx()  

    #
    if index_plot is None:
        index_plot = range(0, steps.shape[-1])
    x_values = steps[index_plot] * deltaT
    
    # angular momentum effective-right
    ax2.plot(x_values, angular_momentum_eff[index_plot],
             color=angular_color, linewidth=angular_linewidth, label=label_angular)
    
    # theta-left
    ax1.plot(x_values, Theta[index_plot], 
             color=theta_color,  linewidth=theta_linewidth, label=label_theta)
    
    #
    if show_zeroline:
        ax1.axhline(0, color='k', linestyle='--', linewidth=1.5, alpha=0.5)
        ax2.axhline(0, color='k', linestyle='--', linewidth=1.5, alpha=0.5)
    
    #
    if show_vlines:
        plot_period_vlines(ax1, x_values, Theta, show_vline=True, show_markline=show_marklines)

    #
    ax1.set_xlabel('Time', fontsize=18)
    if xtick_spacing is not None:
        ax1.xaxis.set_major_locator(ticker.MultipleLocator(xtick_spacing))
    ax1.set_ylabel(label_theta, color=theta_color, rotation=0, fontsize=26, va='center')
    ax2.set_ylabel(label_angular, color=angular_color, rotation=0, fontsize=18, va='center', ha='left')
    
    #
    ax1.tick_params(axis='y', labelcolor=theta_color)
    ax2.tick_params(axis='y', labelcolor=angular_color)
    
    #
    if xlim is not None:
        ax1.set_xlim(xlim)
    if theta_ylim is not None:
        ax1.set_ylim(theta_ylim)
    if angular_ylim is not None:
        ax2.set_ylim(angular_ylim)

    #
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    
    if title_string is not None:
        ax1.set_title(title_string)

    plt.tight_layout()
    plt.savefig(os.path.join(figure_dir, figure_name), dpi=dpi)
    plt.close()

In [54]:
if save_name == "3VarMassUUD_cos_e8ed382a034d5eb0f2787073dba3803d0387b66f4e1f85c2feb492b768fb4711":
    figsize = (6,5)
    strobescope_list = [157500] #[4200, 42000, 105000, 157500, 210000]
    xlim_list =  [[200, 400]]#[[200,230], [200,230], [200,300], [200, 400], [200, 420]]
    xtick_spacing_list = [None] #[None, None, None, None, None]
    theta_ylim_list = [[-11, -5]] #[[-6.1,-4.9], [-6.0,-4.9], [-7.7, -5], [-11, -5], [-11, -5]]
    angular_ylim_list = [[-0.09,-0.06]] #[[-0.076,-0.069], [-0.0745,-0.069], [-0.073,-0.071], [-0.09,-0.06], [-0.073,-0.0725]]
    show_zerolines = [False] #[False, False, False, False, False]
else:
    strobescope_list = []


for i, strobescope in enumerate(strobescope_list):
    print("strobescope={:.4f} [{:>7.1%} of T]".format(strobescope*deltaT, strobescope/stepsLeastCommonPeriod))
    plot_combined_theta_angular(figure_dir, index_plot=range(50*stepsLeastCommonPeriod+1,200*stepsLeastCommonPeriod+1,strobescope),
                                theta_linewidth=3, angular_linewidth=1.5,
                                xlim=xlim_list[i], xtick_spacing=xtick_spacing_list[i],
                                theta_ylim=theta_ylim_list[i], angular_ylim=angular_ylim_list[i],
                                title_string="{:>7.1%}T".format(strobescope/stepsLeastCommonPeriod),
                                show_zeroline=show_zerolines[i], show_vlines=False, show_marklines=False,
                                figure_name="combined_theta_angular_strobe{:d}.png".format(strobescope), figsize=figsize)

strobescope=1.5750 [  75.0% of T]


## Visualization-3: $\psi=\phi_{13}-\phi_{23}$ scan, $\phi_{13}=\pi/4$

In [47]:
from fractions import Fraction

In [48]:
psi_scan_file = os.path.join(".", "data", "psi.out")

psi_scan_file, os.path.exists(psi_scan_file)

('./data/psi.out', True)

In [49]:
if os.path.exists(psi_scan_file):
    with open(psi_scan_file, "r") as file:
        psi_scan_lines = [line.strip().split("\t") for line in file.readlines() if not line.startswith("#")]

    psi_scan_data = [(float(x),float(y)) for (x,y) in psi_scan_lines]

    print(len(psi_scan_lines), len(psi_scan_data))

1001 1001


In [50]:
([Fraction(x/np.pi) for x in np.vectorize(Fraction)(np.linspace(0, 2*np.pi, 9)/np.pi)][0]).numerator == 0

True

### **Figures 1 c)**: $\theta(200T; \psi)$

In [51]:
def plot_psi_scan(figure_dir, psi_scan_data, figsize=(8,5), figure_name="psi_scan.png", dpi=150, 
                  linewidth=3, colors ='r', xticks_num=9):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    
    x_values = [data[0] for data in psi_scan_data]
    y_values = [data[1] for data in psi_scan_data]
    
    #
    ax.axhline(0, color='k', linestyle='--', linewidth=1.5, alpha=0.5)
    
    # \psi = \phi_{13} - \phi_{23}
    ax.plot(x_values, y_values, '-', color=colors, linewidth=linewidth)

    #
    xticks_pos = np.linspace(0, 2*np.pi, xticks_num)
    xticks_labels = []
    for x in np.vectorize(Fraction)(xticks_pos/np.pi):
        numerator = x.numerator
        denominator = x.denominator
        if numerator == 0:
            xticks_labels.append("$0$")
        elif numerator == 1 and denominator == 1:
            xticks_labels.append("$\pi$")
        elif numerator == 1:
            xticks_labels.append("$\pi/{:d}$".format(denominator))
        elif denominator == 1:
            xticks_labels.append("${:d}\pi$".format(numerator))
        else:
            xticks_labels.append("${:d}\pi$".format(numerator)+"/"+"${:d}$".format(denominator))
    ax.set_xticks(xticks_pos, labels=xticks_labels)
    ax.grid(True, linestyle=':', color='gray', alpha=0.7)
    ax.set_xlabel("Relative phase difference", fontsize=18) #ax.set_xlabel("$\\psi = \\phi_{13} - \\phi_{23}$")
    ax.set_ylabel("$\\theta$", rotation=0, fontsize=26)
    
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(os.path.join(figure_dir,figure_name), dpi=dpi)
    plt.close()

In [52]:
if os.path.exists(psi_scan_file):
    plot_psi_scan(figure_dir, psi_scan_data, figsize=(6,5), figure_name="psi_scan.png", xticks_num=5, linewidth=5)

# End of Simulation

In [53]:
print("Save name for run: {:}".format(save_name))
print("Path of data     :{:}".format(os.path.join(save_dir, save_name)))
print("Path of figures  :{:}".format(figure_dir))

Save name for run: 3VarMassUUD_cos_e8ed382a034d5eb0f2787073dba3803d0387b66f4e1f85c2feb492b768fb4711
Path of data     :./data/3VarMassUUD_cos_e8ed382a034d5eb0f2787073dba3803d0387b66f4e1f85c2feb492b768fb4711
Path of figures  :./figures/3VarMassUUD_cos_e8ed382a034d5eb0f2787073dba3803d0387b66f4e1f85c2feb492b768fb4711
